# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/juliencounts/Desktop/Data_Analysis/API/WeatherPy/Starter_Code/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,city,lat,lng,max_Temp,humidity,cloudiness,wind_Speed,country,date
0,0,puerto ayora,-0.7393,-90.3518,75.54,100,97,7.00,EC,2023-04-17 03:57:12
1,1,tarrafal,15.2833,-23.7667,74.19,87,76,15.32,CV,2023-04-17 03:57:13
2,2,sobolevo,54.4299,31.8996,41.95,93,100,4.88,RU,2023-04-17 03:57:13
3,3,dikson,73.5069,80.5464,5.99,99,100,11.07,RU,2023-04-17 03:57:13
4,4,spas-klepiki,55.1347,40.1767,42.46,48,100,3.65,RU,2023-04-17 03:57:14


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'lng',
    'lat',
    geo = True,
    tiles = "OSM",
    frame_width = 400,
    frame_height = 400,
    size = "humidity",
    color = "city"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lng,lat]   (city,humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
trim_weather_df = city_data_df[(city_data_df["max_Temp"]>=21)
                                         & (city_data_df["max_Temp"]<=27)
                                         & (city_data_df["wind_Speed"]<=4.5)
                                         & (city_data_df["cloudiness"]<=1)]

# Drop any rows with null values
trim_weather_df = city_data_df.dropna()
# Display sample data
trim_weather_df.head()

,City_ID,city,lat,lng,max_Temp,humidity,cloudiness,wind_Speed,country,date
0,0,puerto ayora,-0.7393,-90.3518,75.54,100,97,7.00,EC,2023-04-17 03:57:12
1,1,tarrafal,15.2833,-23.7667,74.19,87,76,15.32,CV,2023-04-17 03:57:13
2,2,sobolevo,54.4299,31.8996,41.95,93,100,4.88,RU,2023-04-17 03:57:13
3,3,dikson,73.5069,80.5464,5.99,99,100,11.07,RU,2023-04-17 03:57:13
4,4,spas-klepiki,55.1347,40.1767,42.46,48,100,3.65,RU,2023-04-17 03:57:14


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = trim_weather_df.iloc[: , [1, 8, 2, 3, 5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df.head()

,city,country,lat,lng,humidity,Hotel Name
0,puerto ayora,EC,-0.7393,-90.3518,100,
1,tarrafal,CV,15.2833,-23.7667,87,
2,sobolevo,RU,54.4299,31.8996,93,
3,dikson,RU,73.5069,80.5464,99,
4,spas-klepiki,RU,55.1347,40.1767,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
categories = "accomodation.hotel"
limit= 1
params= {"categories":categories,"limit":limit,"apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["lng"]
    latitude = row["lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'city']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto ayora - nearest hotel: No hotel found
tarrafal - nearest hotel: No hotel found
sobolevo - nearest hotel: No hotel found
dikson - nearest hotel: No hotel found
spas-klepiki - nearest hotel: No hotel found
namibe - nearest hotel: No hotel found
tucuma - nearest hotel: No hotel found
dwarka - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
umba - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
manokwari - nearest hotel: No hotel found
alappuzha - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
hithadhoo - nearest hotel: No hotel found
ilulissat - nearest hotel: No hotel found
jaslo - nearest hotel: No hotel found
qaanaaq - nearest hotel: No hotel found
rikitea - nearest hotel: No hotel found
ponta delgada - nearest hotel: No hotel found
buraydah - nearest hotel: No hotel found
mataura - nearest hotel: No hotel found
tyrma - nearest hotel: No hotel found
leningradskiy - nearest hotel: N

ust-ordynskiy - nearest hotel: No hotel found
faanui - nearest hotel: No hotel found
barranca - nearest hotel: No hotel found
dhidhdhoo - nearest hotel: No hotel found
carballo - nearest hotel: No hotel found
khani - nearest hotel: No hotel found
lac du bonnet - nearest hotel: No hotel found
zhigansk - nearest hotel: No hotel found
uromi - nearest hotel: No hotel found
palmer - nearest hotel: No hotel found
sao sebastiao do cai - nearest hotel: No hotel found
alice springs - nearest hotel: No hotel found
joshimath - nearest hotel: No hotel found
nantucket - nearest hotel: No hotel found
necochea - nearest hotel: No hotel found
rorvik - nearest hotel: No hotel found
port-gentil - nearest hotel: No hotel found
manaure - nearest hotel: No hotel found
kropotkin - nearest hotel: No hotel found
ormara - nearest hotel: No hotel found
mustafakemalpasa - nearest hotel: No hotel found
maiduguri - nearest hotel: No hotel found
saltillo - nearest hotel: No hotel found
oksfjord - nearest hotel: No 

behbahan - nearest hotel: No hotel found
cuamba - nearest hotel: No hotel found
hualmay - nearest hotel: No hotel found
galesong - nearest hotel: No hotel found
castanhal - nearest hotel: No hotel found
coos bay - nearest hotel: No hotel found
charters towers - nearest hotel: No hotel found
baiao - nearest hotel: No hotel found
curup - nearest hotel: No hotel found
goulburn - nearest hotel: No hotel found
dongsheng - nearest hotel: No hotel found
ukiah - nearest hotel: No hotel found
awjilah - nearest hotel: No hotel found
adrar - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
panacan - nearest hotel: No hotel found
pangkalanbuun - nearest hotel: No hotel found
cairns - nearest hotel: No hotel found
eureka - nearest hotel: No hotel found
tumaco - nearest hotel: No hotel found
altamira - nearest hotel: No hotel found
oussouye - nearest hotel: No hotel found
thompson - nearest hotel: No hotel found
bowen - nearest hotel: No hotel found
los llanos de aridane - 

,city,country,lat,lng,humidity,Hotel Name
0,puerto ayora,EC,-0.7393,-90.3518,100,No hotel found
1,tarrafal,CV,15.2833,-23.7667,87,No hotel found
2,sobolevo,RU,54.4299,31.8996,93,No hotel found
3,dikson,RU,73.5069,80.5464,99,No hotel found
4,spas-klepiki,RU,55.1347,40.1767,48,No hotel found
...,...,...,...,...,...,...
549,mbandaka,CD,0.0487,18.2603,98,No hotel found
550,alto araguaia,BR,-17.3147,-53.2153,87,No hotel found
551,jamiltepec,MX,16.2833,-97.8167,79,No hotel found
552,arlit,NE,18.7369,7.3853,14,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "lng",
    "lat",
    geo = True,
    tiles = "OSM",
    frame_width = 400,
    frame_height = 400,
    size = "humidity",
    scale = 0.8,
    color = "city",
    hover_cols=["country", "Hotel Name"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lng,lat]   (city,humidity,country,Hotel Name)